# Supertracker Introduction

This notebook demonstrates how to use the supertracker library for multi-object tracking.

## Installation

In [ ]:
!pip install -q supertracker ultralytics opencv-python

## Basic Usage with YOLO

In [ ]:
import cv2
from ultralytics import YOLO
from supertracker import ByteTrack
from supertracker import Detections

# Initialize YOLO and tracker
model = YOLO('yolov8n.pt')
tracker = ByteTrack(
    track_activation_threshold=0.25,
    lost_track_buffer=30,
    frame_rate=30
)

## Process Single Image

In [ ]:
# Load and process single image
image = cv2.imread('example.jpg')
results = model(image, verbose=False)[0]

# Convert YOLO results to Detections format
detections = Detections(
    xyxy=results.boxes.xyxy.cpu().numpy(),
    confidence=results.boxes.conf.cpu().numpy(),
    class_id=results.boxes.cls.cpu().numpy().astype(int)
)

# Update tracker
tracked_objects = tracker.update_with_detections(detections)

# Visualize results
for i in range(len(tracked_objects)):
    box = tracked_objects.xyxy[i].astype(int)
    track_id = tracked_objects.tracker_id[i]
    class_id = tracked_objects.class_id[i]
    conf = tracked_objects.confidence[i]
    
    # Draw bounding box
    cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
    
    # Draw label
    label = f"#{track_id} {model.names[class_id]} {conf:.2f}"
    cv2.putText(image, label, (box[0], box[1]-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

cv2.waitKey(0)

## Process Video Stream

In [ ]:
def process_video(source=0):  # 0 for webcam
    cap = cv2.VideoCapture(source)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Run YOLO detection
        results = model(frame, verbose=False)[0]
        
        # Convert to Detections format
        detections = Detections(
            xyxy=results.boxes.xyxy.cpu().numpy(),
            confidence=results.boxes.conf.cpu().numpy(),
            class_id=results.boxes.cls.cpu().numpy().astype(int)
        )
        
        # Update tracker
        tracked_objects = tracker.update_with_detections(detections)
        
        # Visualize results
        for i in range(len(tracked_objects)):
            box = tracked_objects.xyxy[i].astype(int)
            track_id = tracked_objects.tracker_id[i]
            class_id = tracked_objects.class_id[i]
            conf = tracked_objects.confidence[i]
            
            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
            label = f"#{track_id} {model.names[class_id]} {conf:.2f}"
            cv2.putText(frame, label, (box[0], box[1]-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

# Run video processing
process_video('vid.mp4')  # or just process_video() for webcam

## Advanced Configuration

In [ ]:
# Example with custom configuration
tracker = ByteTrack(
    track_activation_threshold=0.3,  # Higher threshold for more confident tracks
    lost_track_buffer=45,           # Longer buffer for better occlusion handling
    minimum_matching_threshold=0.85, # Stricter matching for better identity preservation
    frame_rate=30,                  # Match your video frame rate
    minimum_consecutive_frames=2     # Require more frames for track confirmation
)